# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc19c71fca0>
├── 'a' --> tensor([[ 1.2720,  0.6516, -2.4516],
│                   [ 1.1334,  0.3185,  1.2699]])
└── 'x' --> <FastTreeValue 0x7fc19c71f7f0>
    └── 'c' --> tensor([[-1.0283, -0.4688,  1.9659,  2.5100],
                        [ 0.9677,  0.1151,  0.8351,  0.6686],
                        [-1.6604,  1.1215, -0.1914, -0.1482]])

In [4]:
t.a

tensor([[ 1.2720,  0.6516, -2.4516],
        [ 1.1334,  0.3185,  1.2699]])

In [5]:
%timeit t.a

57.8 ns ± 0.0964 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc19c71fca0>
├── 'a' --> tensor([[-0.4986, -0.0550, -0.9742],
│                   [ 1.0265,  0.7553, -0.0452]])
└── 'x' --> <FastTreeValue 0x7fc19c71f7f0>
    └── 'c' --> tensor([[-1.0283, -0.4688,  1.9659,  2.5100],
                        [ 0.9677,  0.1151,  0.8351,  0.6686],
                        [-1.6604,  1.1215, -0.1914, -0.1482]])

In [7]:
%timeit t.a = new_value

57.7 ns ± 0.0193 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.2720,  0.6516, -2.4516],
               [ 1.1334,  0.3185,  1.2699]]),
    x: Batch(
           c: tensor([[-1.0283, -0.4688,  1.9659,  2.5100],
                      [ 0.9677,  0.1151,  0.8351,  0.6686],
                      [-1.6604,  1.1215, -0.1914, -0.1482]]),
       ),
)

In [10]:
b.a

tensor([[ 1.2720,  0.6516, -2.4516],
        [ 1.1334,  0.3185,  1.2699]])

In [11]:
%timeit b.a

51.9 ns ± 0.0421 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 2.0536, -0.1757,  0.0707],
               [ 1.6439,  0.2873, -0.4892]]),
    x: Batch(
           c: tensor([[-1.0283, -0.4688,  1.9659,  2.5100],
                      [ 0.9677,  0.1151,  0.8351,  0.6686],
                      [-1.6604,  1.1215, -0.1914, -0.1482]]),
       ),
)

In [13]:
%timeit b.a = new_value

486 ns ± 0.0746 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

836 ns ± 0.158 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 36.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

141 µs ± 6.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc0e79bf5b0>
├── 'a' --> tensor([[[ 1.2720,  0.6516, -2.4516],
│                    [ 1.1334,  0.3185,  1.2699]],
│           
│                   [[ 1.2720,  0.6516, -2.4516],
│                    [ 1.1334,  0.3185,  1.2699]],
│           
│                   [[ 1.2720,  0.6516, -2.4516],
│                    [ 1.1334,  0.3185,  1.2699]],
│           
│                   [[ 1.2720,  0.6516, -2.4516],
│                    [ 1.1334,  0.3185,  1.2699]],
│           
│                   [[ 1.2720,  0.6516, -2.4516],
│                    [ 1.1334,  0.3185,  1.2699]],
│           
│                   [[ 1.2720,  0.6516, -2.4516],
│                    [ 1.1334,  0.3185,  1.2699]],
│           
│                   [[ 1.2720,  0.6516, -2.4516],
│                    [ 1.1334,  0.3185,  1.2699]],
│           
│                   [[ 1.2720,  0.6516, -2.4516],
│                    [ 1.1334,  0.3185,  1.2699]]])
└── 'x' --> <FastTreeValue 0x7fc0e79bf490>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.7 µs ± 196 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc19c7120d0>
├── 'a' --> tensor([[ 1.2720,  0.6516, -2.4516],
│                   [ 1.1334,  0.3185,  1.2699],
│                   [ 1.2720,  0.6516, -2.4516],
│                   [ 1.1334,  0.3185,  1.2699],
│                   [ 1.2720,  0.6516, -2.4516],
│                   [ 1.1334,  0.3185,  1.2699],
│                   [ 1.2720,  0.6516, -2.4516],
│                   [ 1.1334,  0.3185,  1.2699],
│                   [ 1.2720,  0.6516, -2.4516],
│                   [ 1.1334,  0.3185,  1.2699],
│                   [ 1.2720,  0.6516, -2.4516],
│                   [ 1.1334,  0.3185,  1.2699],
│                   [ 1.2720,  0.6516, -2.4516],
│                   [ 1.1334,  0.3185,  1.2699],
│                   [ 1.2720,  0.6516, -2.4516],
│                   [ 1.1334,  0.3185,  1.2699]])
└── 'x' --> <FastTreeValue 0x7fc19c7127f0>
    └── 'c' --> tensor([[-1.0283, -0.4688,  1.9659,  2.5100],
                        [ 0.9677,  0.1151,  0.8351,  0.6686],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 54.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.9 µs ± 157 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.0283, -0.4688,  1.9659,  2.5100],
                       [ 0.9677,  0.1151,  0.8351,  0.6686],
                       [-1.6604,  1.1215, -0.1914, -0.1482]],
              
                      [[-1.0283, -0.4688,  1.9659,  2.5100],
                       [ 0.9677,  0.1151,  0.8351,  0.6686],
                       [-1.6604,  1.1215, -0.1914, -0.1482]],
              
                      [[-1.0283, -0.4688,  1.9659,  2.5100],
                       [ 0.9677,  0.1151,  0.8351,  0.6686],
                       [-1.6604,  1.1215, -0.1914, -0.1482]],
              
                      [[-1.0283, -0.4688,  1.9659,  2.5100],
                       [ 0.9677,  0.1151,  0.8351,  0.6686],
                       [-1.6604,  1.1215, -0.1914, -0.1482]],
              
                      [[-1.0283, -0.4688,  1.9659,  2.5100],
                       [ 0.9677,  0.1151,  0.8351,  0.6686],
                       [-1.6604,  1.1215, -0.1914, -0.1482]],

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 81.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.0283, -0.4688,  1.9659,  2.5100],
                      [ 0.9677,  0.1151,  0.8351,  0.6686],
                      [-1.6604,  1.1215, -0.1914, -0.1482],
                      [-1.0283, -0.4688,  1.9659,  2.5100],
                      [ 0.9677,  0.1151,  0.8351,  0.6686],
                      [-1.6604,  1.1215, -0.1914, -0.1482],
                      [-1.0283, -0.4688,  1.9659,  2.5100],
                      [ 0.9677,  0.1151,  0.8351,  0.6686],
                      [-1.6604,  1.1215, -0.1914, -0.1482],
                      [-1.0283, -0.4688,  1.9659,  2.5100],
                      [ 0.9677,  0.1151,  0.8351,  0.6686],
                      [-1.6604,  1.1215, -0.1914, -0.1482],
                      [-1.0283, -0.4688,  1.9659,  2.5100],
                      [ 0.9677,  0.1151,  0.8351,  0.6686],
                      [-1.6604,  1.1215, -0.1914, -0.1482],
                      [-1.0283, -0.4688,  1.9659,  2.5100],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 250 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
